<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/%EC%82%AC%EC%9A%A9%EC%9E%90%EC%9D%98_%EC%95%A1%EC%85%98_%EC%88%98_%EC%A7%91%EA%B3%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#권한 설정

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%load_ext google.colab.data_table

# 고객 액션 데이터셋

In [5]:
%%bigquery --project ringed-marker-347607
select *
from ch5.action_log

,session,user_id,action,category,products,amount,stamp
0,989004ea,U001,view,None,None,NaN,2016-11-03 18:00:00
1,989004ea,U001,review,drama,D001,NaN,2016-11-03 18:00:00
2,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
3,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
4,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
5,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
6,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
7,989004ea,U001,add_cart,drama,D002,NaN,2016-11-03 18:01:00
8,989004ea,U001,add_cart,drama,"D001,D002",NaN,2016-11-03 18:02:00
9,47db0370,U002,add_cart,drama,D001,NaN,2016-11-03 19:00:00


# 액션 수와 비율을 계산하는 쿼리

In [7]:
%%bigquery --project ringed-marker-347607
with
stats as (
  --로그 전체의 유니크 사용자 수 구하기
  select count(distinct session) as total_uu
  from `ringed-marker-347607.ch5.action_log`
)
select
l.action,
--액션의 수
count(1) as action_count,
--전체 uu
total_uu,
-- 사용률 : 액션 수 / 액션 UU
100.0 * count(distinct l.session) / s.total_uu as usage_rate,
-- 1인당 액션 수 : 액션 수 / 액션 uu
1.0 * count(1)/count(distinct l.session) as count_per_user
from 
`ringed-marker-347607.ch5.action_log` as l
cross join
stats as s
group by
l.action,s.total_uu

,action,action_count,total_uu,usage_rate,count_per_user
0,view,1,4,25.0,1.000000
1,review,1,4,25.0,1.000000
2,add_cart,12,4,75.0,4.000000
3,favorite,1,4,25.0,1.000000
4,purchase,5,4,75.0,1.666667


# 로그인 사용자와 비로그인 사용자를 구분하여 집계하기

In [8]:
%%bigquery --project ringed-marker-347607
with
action_log_with_status as (
  select 
  session,
  user_id,
  action,
  case when coalesce(user_id,"") <> "" then 'login' else 'guest' end as login_status
  from
  `ringed-marker-347607.ch5.action_log`
)
select *
from action_log_with_status

,session,user_id,action,login_status
0,989004ea,U001,view,login
1,989004ea,U001,review,login
2,989004ea,U001,add_cart,login
3,989004ea,U001,add_cart,login
4,989004ea,U001,add_cart,login
5,989004ea,U001,add_cart,login
6,989004ea,U001,add_cart,login
7,989004ea,U001,add_cart,login
8,989004ea,U001,add_cart,login
9,47db0370,U002,add_cart,login
